In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array
# import string
# from string import punctuation
# from itertools import chain

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_selection import SelectPercentile, chi2, f_regression, f_classif
from sklearn.utils import shuffle
from scipy.stats import pearsonr

! pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
# from sklearn import svm

# nltk.download('stopwords')
# from nltk.stem import WordNetLemmatizer
# from nltk import ngrams

import re
import string

! pip install nltk
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


! pip install wordcloud
from wordcloud import WordCloud, STOPWORDS

! pip install tensorflow

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
import io
import json
from collections import Counter
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
from datetime import datetime
import matplotlib.gridspec as gridspec



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\kaila\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kaila\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaila\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kaila\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Model Predictions

In [ ]:
analysis_set = pd.read_csv("csv's/AnalysisDataset.csv")

In [ ]:
reviews = analysis_set["cleanedText"]
reviews = reviews.apply(lambda x: re.sub(r"\s+", " ", str(x)).strip() if isinstance(x, (str, float, int)) else x)

In [ ]:
unseen_reviews = []
for review in reviews:
    unseen_reviews.append(review)

In [ ]:
with open('embedded.json') as f:
    data = json.load(f)
    loaded_tokenizer = tokenizer_from_json(data)

In [ ]:
unseen_tokenized = loaded_tokenizer.texts_to_sequences(unseen_reviews)
unseen_padded = pad_sequences(unseen_tokenized, padding='post', maxlen=100)

In [ ]:
from keras.models import load_model

model_path = 'lstm_model.keras'
pretrained_lstm_model = load_model(model_path)
pretrained_lstm_model.summary()
preds = pd.Series()
preds = pretrained_lstm_model.predict(unseen_padded)
# unseen_sentiments

In [ ]:
preds = (preds > 0.5).astype(int).flatten()

In [ ]:
analysis_set["predictedSentiment"] = preds
analysis_set.head(1)

In [ ]:
analysis_set.to_csv("csv's/Predictions.csv")

In [61]:
df = pd.DataFrame()
df = pd.read_csv("csv's/Predictions.csv")

In [62]:
positives = df[df['predictedSentiment']==1]
positives.to_csv("csv's/PositiveSentiments.csv")

In [63]:
df = pd.DataFrame()
df = pd.read_csv("csv's/PositiveSentiments.csv")

In [64]:
alpha = 0.7
beta = 0.25
gamma = 0.05

In [65]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year

# Category Popularity
## CPS - Category Popularity Score
This is used to compare positive popularity across categories over time

In [99]:
cps_vals = {}
ppcps_vals = {}
bpcps_vals = {}
ppbps_vals = {}

In [ ]:
for category in df['category'].unique():
    category_data = df[df['category']==category]

    for year in category_data['year'].unique():
        for month in category_data['month'].unique():
            category_by_year_month = category_data[(category_data["year"]==year) & (category_data["month"]==month)]
            
            if not category_by_year_month.empty:
                monthly_total_reviews = category_by_year_month["numRating"].sum()
                monthly_total_average_rating = category_by_year_month["average_rating"].sum()
                monthly_total_votes = category_by_year_month["total_votes"].sum()

                cps = ((alpha * monthly_total_reviews) +
                        (beta * monthly_total_average_rating) +
                        (gamma * monthly_total_votes))
                
                cps_vals[(category, year, month)] = cps

In [69]:
gc.collect()

42473

In [70]:
cps_summary = pd.DataFrame([(cat, yr, mo, val) 
                           for (cat, yr, mo), val in cps_vals.items()],
                           columns=['category', 'year', 'month', 'CPS'])

In [71]:
cps_summary = cps_summary.sort_values(['year', 'month', 'category'])

In [72]:
cps_summary["CPS"] = np.log1p(cps_summary["CPS"])
min_val = cps_summary["CPS"].min()
max_val = cps_summary["CPS"].max()
cps_summary["CPS scaled"] = 0 + (cps_summary["CPS"] - min_val) * (100 - 0) / (max_val - min_val)

In [73]:
cps_summary.head(3)

,category,year,month,CPS,CPS scaled
5657,Digital Music,1998,4,2.953607,11.528284
5661,Digital Music,1998,6,3.045712,12.039228
5658,Digital Music,1998,7,9.140170,45.847677


In [74]:
cps_summary.to_csv("csv's/CPS_Summary.csv")
del cps_summary
gc.collect()

0

# Product popularity 
## PPCPS - Product per Category popularity score
Used to compare products within the same category over time

In [101]:
for category in df['category'].unique():
    category_data = df[df['category']==category]

    for product in category_data['item'].unique():
        product_data = category_data[category_data['item']==product]
        if not product_data.empty:
            prod_total_reviews = product_data["numRating"].sum()
            prod_average_rating = product_data["average_rating"].sum()
            prod_total_votes = product_data["total_votes"].sum()

            if 'timestamp' in product_data.columns and not product_data['timestamp'].empty:
                timestamp = pd.to_datetime(product_data['timestamp'].iloc[0])

                if pd.notna(timestamp):
                    timestamp = pd.to_datetime(timestamp)
                    year = int(timestamp.year)
                    month = int(timestamp.month)
                    # print(f"{year}/{month}")
            
                    ppcps = ((alpha * prod_total_reviews) +
                            (beta * prod_average_rating) + 
                            (gamma * prod_total_votes))
                                
                    ppcps_vals[(category, product, year, month)] = ppcps

In [102]:
ppcps_summary = pd.DataFrame([
    (cat, prod, yr, mo, val) 
    for (cat, prod, yr, mo), val in ppcps_vals.items()
], columns=['category', 'product', 'year', 'month', 'PPCPS'])

In [103]:
ppcps_summary = ppcps_summary.sort_values(['year', 'month', 'category', 'product'])

In [104]:
ppcps_summary["PPCPS"] = np.log1p(ppcps_summary["PPCPS"])
min_val = ppcps_summary["PPCPS"].min()
max_val = ppcps_summary["PPCPS"].max()
ppcps_summary["PPCPS scaled"] = 0 + (ppcps_summary["PPCPS"] - min_val) * (100 - 0) / (max_val - min_val)

In [106]:
ppcps_summary.to_csv("csv's/PPCPS_Summary.csv")

## PPBPS - Product per Brand Popularity score 
Used to compare products across the brand

In [107]:
for category in df['category'].unique():
    category_data = df[df['category']==category]

    for brand in category_data['brand'].unique():
        brand_data = category_data[category_data['brand']==brand]

        for product in brand_data['item'].unique():
            product_data = brand_data[brand_data['item']==product]

            if not product_data.empty:
                prod_total_reviews = product_data["numRating"].sum()
                prod_average_rating = product_data["average_rating"].sum()
                prod_total_votes = product_data["total_votes"].sum()

                if 'timestamp' in product_data.columns and not product_data['timestamp'].empty:
                    timestamp = pd.to_datetime(product_data['timestamp'].iloc[0])

                    if pd.notna(timestamp):
                        timestamp = pd.to_datetime(timestamp)
                        
                        year = int(timestamp.year)
                        month = int(timestamp.month)
            
                        ppbps = ((alpha * prod_total_reviews) +
                                (beta * prod_average_rating) + 
                                (gamma * prod_total_votes))
                                    
                        ppbps_vals[(year, month, category, brand, product)] = ppbps
                

In [108]:
ppbps_summary = pd.DataFrame([(yr, mo, cat, brand, prod, val) 
                           for (yr, mo, cat, brand, prod), val in ppbps_vals.items()],
                           columns=['year', 'month', 'category', 'brand', 'product', 'PPBPS'])

In [109]:
ppbps_summary = ppbps_summary.sort_values(['year', 'month', 'category', 'brand'])

In [110]:
ppbps_summary["PPBPS"] = np.log1p(ppbps_summary["PPBPS"])
min_val = ppbps_summary["PPBPS"].min()
max_val = ppbps_summary["PPBPS"].max()
ppbps_summary["PPBPS scaled"] = 0 + (ppbps_summary["PPBPS"] - min_val) * (100 - 0) / (max_val - min_val)

In [111]:
ppbps_summary['PPBPS scaled'].max(), ppbps_summary['PPBPS scaled'].min()

(np.float64(100.0), np.float64(0.0))

In [112]:
ppbps_summary.to_csv("csv's/PPBPS_Summary.csv")

# Brand Popularity
## BPCPS - Brand per Category Popularity Score
Used to compare brands across a category over time

In [113]:
for category in df["category"].unique():
    category_data = df[df["category"]==category]

    for brand in category_data["brand"].unique():
        brand_data = category_data[category_data['brand']==brand]

        if brand_data.empty:
            continue 

        for year in brand_data['year'].unique():
            for month in brand_data['month'].unique():
                brand_by_year_month = brand_data[(brand_data['year']==year) & (brand_data['month']==month)]

                if not brand_by_year_month.empty:
                    brand_total_reviews = brand_by_year_month['numRating'].sum()
                    brand_total_average_rating = brand_by_year_month['average_rating'].sum()
                    brand_total_votes = brand_by_year_month['total_votes'].sum()

                    bpcps = ((alpha * brand_total_reviews) +
                             (beta * brand_total_average_rating)+ 
                             (gamma * brand_total_votes))

                    bpcps_vals[(category, brand, year, month)] = bpcps

In [114]:
gc.collect()

4052

In [115]:
bpcps_summary = pd.DataFrame([(cat, br, yr, mo, val) 
                           for (cat, br, yr, mo), val in bpcps_vals.items()],
                           columns=['category', 'brand', 'year', 'month', 'BPCPS'])

In [116]:
bpcps_summary = bpcps_summary.sort_values(['category', 'brand', 'year', 'month'])

In [117]:
bpcps_summary["BPCPS"] = np.log1p(bpcps_summary["BPCPS"])
min_val = bpcps_summary["BPCPS"].min()
max_val = bpcps_summary["BPCPS"].max()
bpcps_summary["BPCPS scaled"] = 0 + (bpcps_summary["BPCPS"] - min_val) * (100 - 0) / (max_val - min_val)

In [118]:
bpcps_summary.to_csv("csv's/BPCPS_Summary.csv")